# GRAG - a toy example

Uncomment and install if you are missing a library.

In [1]:
# !pip install -qU \
#     nemoguardrails==0.4.0 \
#     chromadb==0.4.10 \
#     openai==0.27.8 \
#     tqdm==4.65.0

Let's import the necessary modules

In [3]:
import chromadb
import openai
from chromadb.utils import embedding_functions
import os
import getpass
from tqdm.notebook import tqdm
from nemoguardrails import LLMRails, RailsConfig

Provide your OpenAI API key (https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key):

In [4]:
openai.api_key = getpass.getpass("OpenAI API Key:")

OpenAI API Key:········


In [5]:
os.environ["OPENAI_API_KEY"] = openai.api_key

Gloabal model settings for vectorization and chat model. You can change to others, but chances are you will need to adjust the code a bit.

Also keep in mind that you will be charged for using the OpenAI API.

In [6]:
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_ENGINE = "gpt-3.5-turbo"

Data was copied from https://www.ikea.com/pl/pl/customer-service/faq/

Take into account that they may change there and are not updated here.

**Also, this is done without the knowledge of the store owner and is for educational purposes only.**

In [7]:
# C stands for customer and A stands for store agent.
FAQ_DATA = {
    "IKEA": [ # https://www.ikea.com/pl/pl/customer-service/faq/
        [
            "C: Czemu ten produkt jest tak długo niedostępny?",
            """A:Problemy z dostępnością niektórych produktów są bezpośrednim następstwem pandemii.
            Skutki czasowego zachwiania naszego standardowego łańcucha dostaw (produkcji oraz logistyki), mogą być odczuwalne przez dość długi czas, nawet kilka miesięcy. Cały czas mocno pracujemy nad tym, aby przywrócić oczekiwaną dostępność wszystkich produktów, jednak nie zawsze jesteśmy w stanie określić, kiedy dany artykuł powróci do sprzedaży. Przepraszamy i prosimy o cierpliwość.
            Zachęcamy też do bieżącego sprawdzania stanów magazynowych na stronie IKEA.pl, korzystania z opcji ustawienia powiadomienia o dostępności, a także zapoznania się z innymi interesującymi produktami z naszej oferty.""",
        ],
        [
            "C: Dlaczego są takie braki?",
            """A: Ze względu na wpływ pandemii COVID-19 obecnie doświadczamy opóźnień w dostawach, co może mieć wpływ na dostępność produktów w sklepach i w Internecie.
            Każdorazowo, przed wizytą w sklepie, warto sprawdzić dostępność produktów, które chcemy zakupić. Aby to sprawdzić, podczas przeglądania strony produktu, należy kliknąć w link podany w sekcji "stan magazynowy". Jeśli chcesz otrzymywać aktualizacje, możesz wybrać opcję powiadamiania o dostawie i dostępności. Aktualizacje stanów magazynowych są dokonywane co 24 godziny i mogą ulec zmianie z uwagi na wysokie zainteresowanie tymi produktami. Ze względu na aktualizacje systemu i wysoki popyt, istnieje szansa, że pozycja z niskim stanem magazynowym może nie być dostępna w sprzedaży.""",
        ],
        [
            "C: Jakie kroki podejmuje IKEA, aby rozwiązać problem z dostępnością produktów?",
            """A: Aby zapewnić swoim Klientom stałą dostępność produktów, zdecydowaliśmy się globalnie na podjęcie nadzwyczajnych kroków, w tym na zakup własnych kontenerów i czarter dodatkowych statków. Nieustannie planujemy kolejne działania, mające na celu załagodzenie obecnej sytuacji, wywołanej ograniczeniami w transporcie oceanicznym oraz wzrostem liczby zamówień. Współpracujemy z naszymi dostawcami i jesteśmy z nimi w stałym kontakcie, aby opracować najlepsze możliwe rozwiązania, zarówno na chwilę obecną, jak i na przyszłość.""",
        ],
        [
            "C: Kiedy sytuacja wróci do normy?",
            """A: Jest to wyzwanie dla całej branży i trudno nam je przewidzieć. W IKEA ciężko pracujemy, by jak najszybciej udostępnić Klientom, jak najwięcej naszych produktów.
            Kiedy wybrane przeze mnie produkty będą ponownie dostępne?
            Pracujemy w ramach naszego łańcucha dostaw i robimy wszystko, co w naszej mocy, aby jak najszybciej udostępnić wam jak najwięcej produktów IKEA. Zachęcamy do korzystania z możliwości powiadomienia o przywrócenia dostaw i dostępności produktów za pośrednictwem IKEA.pl.""",
        ],
        [
            "C: W jaki sposób mogę się dowiedzieć o dostępności zamawianych przeze mnie produktów?",
            """A: Ze względu na wpływ pandemii COVID-19 obecnie doświadczamy opóźnień w całym łańcuchu dostaw, co ma wpływ na dostępność produktów w sklepach i w Internecie. Klientów zachęcamy każdorazowo do sprawdzenia dostępności produktu na IKEA.pl przed wizytą w sklepie lub zakupem online. W przypadku niedostępności produktu proponujemy do skorzystania z opcji powiadomienia smsem lub mailem o jego dostawie.
            Ze względu na aktualizacje systemu co 24 godziny i wysoki popyt, istnieje szansa, że pozycja z niskim stanem magazynowym może nie być dostępna w sprzedaży lub po dostarczeniu do sklepu szybko się wyprzedać. Dzwoniąc na Infolinię konsultanci korzystają z tego samego źródła wiedzy dostępnego na IKEA.pl.""",
        ],
        [
            "C: Jak sprawdzić dostępność produktu w sklepie stacjonarnym?",
            """A: Najszybciej sprawdzisz ją na stronie interesującego cię produktu.
            Dostępność produktu możesz sprawdzić na naszej stronie internetowej, na stronie produktu, który wpadł ci w oko.
            Zakładkę z dostępnością znajdziesz zaraz obok lub pod zdjęciami produktu (w zależności od tego, czy korzystasz z komputera czy urządzenia mobilnego) - poznasz ją po prostej ikonce sklepu. Obok zobaczysz dostępność artykułu w domyślnie wybranym sklepie - jeśli interesuje cię inna lokalizacja, kliknij w jego nazwę i skorzystaj zopcji "Sprawdź inne sklepy IKEA".
            Jeśli upatrzony przez ciebie produkt jest obecnie niedostępny w wybranym sklepie, zobaczysz też informację o przewidywanej dacie ponownej dostępności.Skorzystaj z opcji "Powiadom mnie, gdy produkt będzie dostępny", a my prześlemy ci informację mailem lub SMS-em, gdy produkt znów będzie w sklepie.
            Instrukcję, jak sprawdzić dostępność produktu zobaczysz na stronie internetowej.""",
        ],
        [
            "C: Jak sprawdzić dostępność produktu w sprzedaży online?",
            """A: Produkty, które możesz kupić online z dostawą oznaczone są informacją "Produkt dostępny do zamówienia". Ichdostępność sprawdzisz w koszyku.
            Jeśli na stronie produktu widoczny jest przycisk "Dodaj do koszyka", a poniżej informacja "Produkt dostępny do zamówienia", oznacza to, że produkt znajduje się w naszej ofercie online z dostawą. Dostępność produktu w naszym magazynie centralnym zostanie zweryfikowana w dalszych krokach, gdy dodasz go do koszyka oraz podasz swój kod pocztowy.
            W przypadku, gdy produkt okaże się niedostępny w magazynie centralnym, możesz sprawdzić jego dostępność w najbliższym stacjonarnym sklepie IKEA i zamówić go z odbiorem własnym w sklepie lub Punkcie Odbioru obsługiwanym przez ten sklep. Na stronie produktu, sprawdź jego dostępność w sklepie, w którym chcesz odebrać zamówienie lub w sklepie obsługującym dany Punktu Odbioru - ich listę znajdziesz tutaj: Punkty Odbioru Zamówień.
            Jeśli przy produkcie znajduje się wyłącznie przycisk "Dodaj do listy zakupów"  oraz informacja "Produkt niedostępny do zamówienia", oznacza to, że jest to produkt, który możesz zakupić wyłącznie w sklepie stacjonarnym. Dotyczy to np. żywych roślin czy tkanin na metry.
            Instrukcję, jak sprawdzić dostępność produktu zobaczysz na stronie internetowej.""",
        ],
        [
            "C: Ile kosztuje dostawa do domu?",
            """A: Koszt dostawy do domu zależy od wielkości Twoich zakupów i zaczyna się już od 15 zł.
            Zawsze staramy się zaproponować ci najkorzystniejszy koszt dostawy. Jeśli to tylko możliwe, zrealizujemy dostawę kurierem w cenie od 15 zł. Koszt dostawy do domu firmą transportową (dla większych zamówień, np. z meblami) zaczyna się od 69 zł i dostępna jest wtrzech opcjach:
            Dostawa bez wniesienia w cenie od 69 zł (zależy od łącznej wagi produktów, która nie może przekroczyć 200 kg). Zamówienie zostanie dostarczone pod wskazany adres (do pierwszej przeszkody architektonicznej) - odbierzesz je przy samochodzie dostawczym. Termin dostawy wybierzesz podczas składania zamówienia. Obowiązuje całodziennyprzedział czasowy: 8-20.
            Dostawa z wniesieniem w cenie od 109 zł (zależy od łącznej wagi produktów). Zamówienie zostanie wniesione do wskazanego pomieszczenia w Twoim domu. Termin dostawy wybierzesz podczas składania zamówienia. Obowiązuje całodzienny przedział czasowy: 8-20.
            Dostawa z wniesieniem o wybranej porze dnia w cenie od 129 zł (zależy od łącznej wagi produktów). Zamówienie zostanie wniesione do wskazanego pomieszczenia w Twoim domu. Termin dostawy wybierzesz podczas składania zamówienia wraz z jednym z czterech przedziałów czasowych: 8-11, 11-14, 14-17, 17-20. Opcja dostępna jest wmiastach, w których znajdują się sklepy IKEA.
            Kalkulator ceny poszczególnych usług w zależności od wagi oraz listę lokaliazacji, w których są dostępne znajdziesz tutaj.
            Koszt dostawy może zmieniać się wraz z dodawaniem kolejnych produktów do zamówienia. Liczba lub waga produktów możeczasami także wymagać podzielenia zamówienia na kilka przesyłek.""",
        ],
        [
            "C: Jakie opcje dostawy są dostępne w IKEA?",
            """A: Wybierz najwygodniejszą opcję dla siebie - dostawę do domu lub odbiór własny.
            Obecnie w IKEA posiadamy kilka opcji dostawy, więc możesz wybrać tę z nich, która najbardziej ci odpowiada.
            Dostawa kurierem do domu to usługa transportu dla niedużych produktów. Jej koszt to 15 zł za jedną paczkę (do 30 kg). Dostawa kurierem jest dostępna tylko dla niektórych produktów. Informację o tym, czy dany produkt jest nią objęty, znajdziesz w koszyku podczas tworzenia zamówienia online. Przesyłki realizuje firma GLS i dostarcza je do Twoich drzwi. Możesz również wybrać jeden z 1700 punktów odbioru GLS. Średni czas realizacji to 2-4 dni.
            Dostawa firmą transportową do domu to usługa dostawy mebli i akcesoriów. Jej koszt zależy od łącznej wagi produktów i wybranej opcji:
            Dostawa bez wniesienia w cenie od 69 zł (zależy od łącznej wagi produktów, która nie może przekroczyć 200 kg). Zamówienie zostanie dostarczone pod wskazany adres (do pierwszej przeszkody architektonicznej) - odbierzesz je przy samochodzie dostawczym. Termin dostawy wybierzesz podczas składania zamówienia. Obowiązuje całodziennyprzedział czasowy: 8-20.
            Dostawa z wniesieniem w cenie od 89 zł (zależy od łącznej wagi produktów). Zamówienie zostanie wniesione do wskazanego pomieszczenia w twoim domu. Termin dostawy wybierzesz podczas składania zamówienia. Obowiązuje całodzienny przedział czasowy: 8-20.
            Dostawa z wniesieniem o wybranej porze dnia w cenie od 129 zł (zależy od łącznej wagi produktów). Zamówienie zostanie wniesione do wskazanego pomieszczenia w twoim domu. Termin dostawy wybierzesz podczas składania zamówienia wraz z jednym z czterech przedziałów czasowych: 8-11, 11-14, 14-17, 17-20. Opcja dostępna jest wmiastach, w których znajdują się sklepy IKEA.
            Usługa Zamów i odbierz umożliwia odbiór przygotowanego zamówienia w Punkcie Odbioru IKEA zlokalizowanym w/przy sklepie IKEA lub w wielu innych miejscowościach. Preferowany termin odbioru wybierzesz podczas składania zamówienia. Koszt usługi to 29 zł. Aby z niej skorzystać, sprawdźdostępność produktów w sklepie, w którym chcesz odebrać zamówienie lub w sklepie przypisanym do danego Punktu Odbioru. Zobacz listę dostępnych Punktów Odbioru.
            Możesz skorzystać także z odbioru w Paczkomatach InPost. Jest to forma dostawy dla mniejszcych paczek do 25 kg, a jej koszt to 12 zł. Informację o tym, czy dany produkt jest objęty dostawą do Paczkomatu znajdziesz w koszyku.
            Obecnie, wprowadzamy również nową opcję dostawy: do Skrytek IKEA znajdujących się przy naszych sklepach (za 29 zł). Jeśli jest ona dostępna dla twojej lokalizacji, wyświetli się w koszyku po wpisaniu kodu pocztowego.
            Z uwagi na niedostępność produktów lub wielkość zamówienia, nie wszystkie opcje dostawy mogą wyświetlać się w twoim koszyku.
            Jeśli zamawiasz wiele produktów, twoje zamówienie może zostać podzielone na kilka przesyłek, a nawet na kilka form dostawy.""",
        ],
        [
            "C: Jak sprawdziś satus mojego zamówienia",
            """A: W tym celu udaj się do zakładki „Śledź zamówienie”. Sprawdzisz tam aktualny stan zamówienia, planowaną datę dostawy orazogólne informacje o zamówieniu.
            Logując się na swoje konto, uzyskasz możliwość zarządzania zamówieniem.
            Możesz tego dokonać klikając w ikonę samochodu, znajdującą się w prawym górnym rogu strony.""",
        ],
        [
            "C: Dostawa nie przyjechała - co robić?",
            """A: Możliwych jest kilka scenariuszy - sprawdź, która sytuacja dotyczy ciebie i jak możemy pomóc.
            W pierwszej kolejności kliknij tutaj, aby sprawdzić aktualny status dostawy twojego zamówienia i potwierdzić, że jest już ono na etapie dostawy.
            Ważny jest także rodzaj wybranej przez ciebie dostawy:
            dostawa do domu - robimy wszystko, aby twoje zamówienie dostarczyć w terminie, lecz niestety zdarzają się sytuacje(np. korki, wypadki na drodze), przez które kierowca może nie zmieścić się z dojazdem w planowanym oknie czasowym.Poprosimy cię o jeszcze trochę cierpliwości. Jeśli minęło już kilka godzin, skontaktuj się z nami telefonicznie lub na czacie.
            dostawa kurierem - pamiętaj, że dla przesyłek kurierskich, data realizacji dostawy wybrana podczas składania zamówienia jest orientacyjna - o ostatecznym terminie doręczenia firma GLS poinformuje cię w osobnej wiadomości, gdy odbierze paczkę dla ciebie. Jeśli paczka nie dotarła, mimo potwierdzenia od firmy kurierskiej, skontaktuj się bezpośrednio z GLS - firma zweryfikuje, gdzie znajduje się przesyłka.
            Jeśli twoje zamówienie miało do ciebie dotrzeć nie dziś, a już w poprzednich dniach, zadzwoń na numer 22 275 00 00 lub porozmawiaj z naszym pracownikiem na czacie - ustalimy, co się wydarzyło i co możemy zrobić w tej sytuacji.""",
        ],
        [
            "C: Jak mogę otrzymać fakturę za zakupy w IKEA?",
            """A: To zależy od sposobu, w jaki robisz swoje zakupy, ale jedno jest pewne - to nic trudnego.
            Robiąc zakupy w sklepie stacjonarnym, fakturę do paragonu z kasy możesz wystawić samodzielnie w kiosku Szybka faktura przy wyjściu ze sklepu (masz na to czas do 90 dni od zakupu, licząc od pierwszego dnia kolejnego miesiąca po zakupie) -potrzebny ci będzie oryginał paragonu.
            Z oryginałem paragonu możesz też udać się do Biura Obsługi Klienta lub wysłać go pocztą na adres sklepu z dopiskiem „Biuro Obsługi Klienta”. Fakturę zamówioną drogą pocztową odeślemy na wskazany adres nadawcy.
            Jeśli faktura wystawiana jest na firmę, paragon musi zawierać numer NIP nabywcy (podany pracownikowi na kasie lub wprowadzony samodzielnie w kasie samoobsługowej). Pamiętaj także, że paragon z NIP o wartości do 450 zł traktowany jest jako faktura uproszczona i nie są wystawione do niego żadne dodatkowe dokumenty.
            Faktura imienna może zostać wystawiona na podstawie paragonu bez numeru NIP.
            Dla zamówień online, faktura (imienna lub na firmę) jest domyślnym dowodem zakupu, który otrzymasz mailowo po opłaceniu zakupów. Dane do faktury podaj podczas składania zamówienia.
            Jeśli chcesz otrzymać fakturę na firmę, wybierz w koszyku opcję "Firma" zamiast "Osoba fizyczna" oraz podaj dane firmy, obowiązkowo z numerem NIP. Jeśli nie widzisz opcji wyboru firmy podczas podawania danych (a tym samym pola umożliwiającego podanie numeru NIP), wyloguj się ze swojego konta IKEA.""",
        ],
        [
            "C: Jak poprawić błędne dane na fakturze?",
            """A: Każdemu może zdarzyć się pomyłka. Wystarczy, że prześlesz do nas notę korygującą.
            Jeżeli w podanych wcześniej danych zauważysz literówkę lub chcesz poprawić nazwę/adres swojej firmy, prześlij do nas wypełnioną i podpisaną notę korygującą na adres: info.ikea.pl@ikea.com. Podpiszemy ją i odeślemy do ciebie, tak aby można ją było dołączyć do pierwotnego dokumentu. To wystarczy.
            Opis i przykładowe, bezpłatne wzory noty korygującej bez problemu znajdziesz w Internecie.
            Pamiętaj, że nota korygująca nie daje możliwości całkowitej zmiany nazwy nabywcy, na którego została wystawiona (np. zosoby fizycznej na firmę lub z jednej firmy na inną).
            Otrzymaną od IKEA notę korygującą należy dołączyć do oryginału faktury, na którym widnieją błędy. Taki komplet dokumentów jest wystarczający w celu uznania korekty: nowa faktura nie zostanie wystawiona.
            Natomiast w przypadku, gdy chcesz zmienić dane z firmowych na prywatne (osoby fizycznej nie prowadzącej działalności gospodarczej), dołącz do noty także oświadczenie o nierozliczeniu przez firmę pierwotnie wystawionej faktury.""",
        ],
        [
            "C: Czy mogę otrzymać fakturę VAT na wybrane pozycje z paragonu lub zamówienia?",
            """A: Masz taką możliwość, jeśli robisz zakupy w sklepie, a twój paragon nie jest fakturą uproszczoną.
            Jeśli twój paragon zawiera numer NIP, a równocześnie jego wartość przekracza 450 zł, możesz skorzystać z kiosku Szybka faktura przy wyjściu ze sklepu. Wydrukujesz w nim fakturę FV oraz będziesz mieć opcję wyboru, które z pozycji na paragonie mają znaleźć się na fakturze.
            Dla pozycji nieuwzględnionych na fakturze, otrzymasz paragon zastępczy (z numerem NIP wskazanym na oryginalnym paragonie). Paragon zastępczynie jest dokumentem fiskalnym i nie może być użyty do wystawienia kolejnej faktury.
            Nie ma możliwości usunięcia wybranych pozycji z faktury uproszocznej, czyli paragonu z numerem NIP o wartości do 450 zł. Pamiętaj jednak,że do swojej dokumentacji firmowej możesz dołączyć taki dokument, rozliczając z niego jedynie wybrane pozycje - wystarczy odpowiednia adnotacja na dokumencie.
            W przypadku zamówień internetowych i telefonicznych, do każdego zamówienia wystawiana jest faktura na wszystkie jego pozycje.""",
        ],
        [
            "C: Czy mogę otrzymać duplikat faktury?",
            """A: Tak, aby otrzymać taki duplikat, napisz do nas lub udaj się się do sklepu IKEA.
            Aby otrzymać duplikat swojej faktury, skontaktuj się z nami w wybrany przez siebie sposób:
            udaj się do Działu Obsługi Klienta w sklepie IKEA,
            napisz do nas na czacie na IKEA.pl/kontakt (w niektórych przypadkach pracownik poprosi cię o napisanie maila),
            wyślij do nas wiadomość na adres: info.ikea.pl@ikea.com.W każdym z przypadków podaj nam istotne dane: datę i kwotę zakupu lub numer zamówienia online, numer NIP nabywcy (lub imię i nazwisko, jeśli chodzi o fakturę imienną) oraz, w miarę możliwości, numer faktury.
            Powyższe zasady dotyczą również faktury uproszczonej, czyli paragonu z numerem NIP o wartości do 450 zł. Duplikat będzie zawierał te same dane, co oryginalny dokument, choć może być wystawiony w innym formacie (A4). Aby uzyskać duplikat faktury uproszczonej, poproś klienta o datę zakupu, sklep, numer paragonu, numer kasy.""",
        ],
        [
            "C: Jak mogę zwrócić zakupiony produkt?",
            """A: Tak, masz na to 365 dni. Dowiedz się, jak to zrobić.
            Jeśli zmienisz zdanie lub Twój zakup okaże się nietrafiony, nie martw się. Produkty kupione w IKEA w Polsce, zarówno w sklepach stacjonarnych, jak i zdalnie, możesz zwrócić w ciągu 365 dni od daty ich wydania/dostawy.
            Aby dokonać zwrotu, odwiedź najbliższy sklep IKEA. Oprócz produktu, zabierz ze sobą dowód zakupu.
            Pamiętaj też, że jest kilka wyjątków. Zwrotowi w IKEA nie podlegają:
            Karty upominkowe i refundacyjne IKEA
            rozpakowane materace (możesz taki materac wymienić w ciągu 90 dni od zakupu zgodnie z zasadami Pokochaj albo zmień)
            produkty na indywidualne zamówienie (np. blaty, panele ścienne)
            rośliny
            tkaniny cięte oraz produkty zakupione z usługą szycia/haftowania
            artykuły spożywcze
            produkty zakupione w Dziale Sprzedaży Okazyjnej
            produkty zmodyfikowane przez klienta
            produkty zakupione w ramach umowy leasingowej
            wartość rabatu udzielonego w ramach promocji
            Wszystkie zasady zwrotów w IKEA znajdziesz w Regulaminie zwrotów IKEA, dostępnym na stronie regulaminy.""",
        ],
        [
            "C: Czy mogę dokonać zwrotu produktu bez paragonu lub faktury?",
            """A: Aby dokonać zwrotu, potrzebować będziesz dowodu zakupu.
            W IKEA możesz dokonać zwrotu zakupionych produktów w ciągu 365 dni od ich wydania/dostawy, posługując się dowodem zakupu w postaci:
            paragonu,
            faktury VAT (w formie elektronicznej lub papierowej),
            karty IKEA Family (wydanej w Polsce), jeśli zakup został na niej zarejestrowany.
            Jeśli nie posiadasz żadnego z wymienionych dowodów zakupu, nie będziemy mogli przyjąć twojego zwrotu.
            Oczywiście, zabierz ze sobą także produkt, który chcesz zwrócić - może on być rozpakowany, a nawet zmontowany, lecz musi być nieużywany. Stan produktu zostanie zweryfikowany przez pracownika sklepu.
            Szczegółowe informacje znajdziesz w obowiązującym Regulaminie zwrotów IKEA, dostępnym na stronie regulaminy.""",
        ],
        [
            "C: Dlaczego reklamację mogę złożyć tylko osobiście lub poprzez kontakt z chatbotem/pracownikiem Domolinii, a nie np. za sprawą e-formularza?",
            """A: Formularz został wyłączony by przyśpieszyć i poprawić jakość procesu składania reklamacji. Zachęcamy do korzystania z Infolinii oraz chatbota znajdującego się na IKEA.pl – w ten sposób, dzięki bezpośredniemu kontaktowi z naszym Działem Wsparcia Klienta, możemy szybciej i skuteczniej zebrać wszelkie informacje dotyczące pojawiających się pytań i spróbować pomóc.""",
        ],
        [
            "C: Dlaczego Infolinia jest ciągle zajęta? Oczekiwanie na połączenie jest wydłużone (np. ponad 25 osób w kolejce).",
            """A: Z uwago na zwiększoną liczbę zamówień aktualnie doświadczamy większego zapotrzebowania na kontakt z pracownikiem i dokładamy wszelkich starań, aby wspierać klientów IKEA w możliwie jak najlepszy sposób.
            Bezpośredni kontakt poprzez infolinię lub chatbota pozwala nam szybciej zebrać całość informacji potrzebnych do rozpatrzenia każdej zgłaszanej do nas sprawy.""",
        ],
        [
            "C: Jak mogę złożyć reklamację?",
            """A: Zobacz możliwe sposoby złożenia reklamacji i wybierz najwygodniejszy dla siebie.
            Chcemy, by produkty oraz usługi IKEA zawsze spełniały oczekiwania naszych klientów i zmieniały ich codzienne życie na lepsze. Dlatego, jeśli coś poszło nie tak, zachęcamy cię do złożenia reklamacji. Postaramy się rozwiązać twój problem tak szybko, jak to możliwe.
            Reklamację możesz zgłosić najwygodniejszą dla siebie drogą:
            online, korzystając z naszego czatu - znajdziesz go w zakłądce kontakt,
            kontaktując się z naszą infolinią Domolinia IKEA pod numerem telefonu 22 275 00 00,
            odwiedzając Dział Obsługi Klienta w dowolnym sklepie stacjonarnym IKEA.
            Zachęcamy cię do skorzystania zwłaszcza z pierwszej opcji - formularza online, który został przygotowany tak, aby w intuicyjny sposób poprowadzić cię przez proces i by w twoim zgłoszeniu niczego nie brakowało.""",
        ],
        [
            "C: Nikt się ze mną nie skontaktował w sprawie reklamacji, co teraz?",
            """A: Jeśli nie minęło jeszcze 14 dni od złożenia reklamacji, poczekaj na kontakt z naszej strony.
            Jeśli czekasz na wieści dotyczące swojej reklamacji, prosimy cię o trochę cierpliwości - informację o decyzji otrzymasz tak szybko, jak to możliwe, a maksymalnie w ciągu 14 dni kalendarzowych od momentu zarejestrowania zgłoszenia.
            Jeśli minęło już 14 dni, a taki kontakt wciąż nie nastąpił:
            upewnij się, czy wiadomość od nas nie trafiła do folderu SPAM w Twojej skrzynce mailowej,
            skontaktuj się z nami na czacie, który znajdziesz na stronie IKEA.pl/kontakt - wyjaśnimy, co się stało.""",
        ],
        [
            "C: Jak mogę sprawdzić status swojej reklamacji?",
            """A: Najszybciej zrobisz to korzystając z naszego czatu.
            Gdy tylko decyzja w twojej reklamacji zostanie podjęta, na pewno się z tobą skontaktujemy.
            Jeśli chcesz jednak upewnić się, na jakim etapie jest twoja sprawa, skorzystaj z czatu z nami na stronie IKEA.pl/kontakt. Nasz czatbot sprawdzi status twojej reklamacji wyszukując ją po numerze.
            W razie potrzeby, na czacie porozmawiasz także z pracownikiem IKEA.""",
        ],
        [
            "C: Czego potrzebuję, aby zgłosić reklamację?",
            """A: Przede wszystkim, będzie to dowód zakupu.
            Elementy, które są nam niezbędne do rozpatrzenia reklamacji to:
            dowód zakupu (np. paragon, faktura, numer karty IKEA Family z zarejestrowanym zakupem, czy potwierdzenie płatności)
            Twoje dane osobowe: imię i nazwisko, adres, numer telefonu i adres mailowy do kontaktu oraz adres, pod którym znajduje się reklamowany produkt,
            informacja, z jakiej podstawy prawnej korzystasz: rękojmi czy gwarancji.
            dokładny opis wady / usterki produktu lub opis nieprawidłowości w wykonaniu usługi.
            Brak któregokolwiek z powyższych sprawi, że będziemy zmuszeni zamknąć Twoje zgłoszenie bez rozpatrzenia, prosząc Cię o uzupełnienie informacji.
            Do zgłoszenia reklamacyjnego warto dołączyć również:
            zdjęcia lub inne materiały multimedialne obrazujące zaistniałą wadę / usterkę lub nieprawidłowości,
            protokół szkody wypełniony dla firmy kurierskiej / transportowej, jeśli uszkodzenie produktu powstało w transporcie i zostało odnotowane w ten sposób,
            protokół z wykonanej usługi dla firmy transportowej, montażowej lub innej, która realizowała reklamowaną usługę,
            w przypadku sprzętów AGD, których instalacja wymaga specjalistycznych uprawnień i które nie były instalowane w ramach usług IKEA: potwierdzenie uprawnień osoby instalującej.
            Takie dodatkowe informacje zazwyczaj pozwalają przyspieszyć rozpatrzenie reklamacji.""",
        ],
        [
            "C: Co zrobić, jeśli nie powiodła się moja płatność za zamówienie?",
            """A: Jeśli twoja płatność za zamówienie internetowe lub telefoniczne się nie udała – z powodu błędu lub przyczyn technicznych – zachęcamy cię do ponownego złożenia zamówienia.
            Zdarza się, że płatność za zamówienie nie powiedzie się. W takiej sytuacji, spróbuj złożyć zamówienie raz jeszcze. Wcześniejsze,nieopłacone zamówienie zostanie automatycznie anulowane w ciągu doby.
            Jeśli nie masz pewności, czy twoja płatność się powiodła, sprawdź swoją skrzynkę mailową. Potwierdzenie złożenia zamówienia powinno zostać przesłane na twojego maila w ciągu 2 h od pomyślnie przeprowadzonej płatności. Upewnij się, że wiadomość od nas nie trafiła do folderu SPAM. Sprawdź również, czy środki zostały pobrane z twojego konta.""",
        ],
        [
            "C: Czy mogę zapłacić za zamówienie przy odbiorze?",
            """A: Płatność pobierana jest zawsze w momencie składania zamówienia – nie oferujemy płatności za pobraniem.
            W IKEA za zakupy, zarówno zdalne, jak i stacjonarne, należy zapłacić w momencie ich dokonywania (wyjątkiem jest płatność ratalna). Nie oferujemy możliwości opłacenia zamówienia przy odbiorze (za pobraniem) ani uzyskania faktury z odroczonym terminem płatności.
            Pamiętaj, że zamówienia internetowe i telefoniczne, które nie zostaną opłacone w ciągu 48 h, zostają anulowane.
            Akceptujemy następujące formy płatności:
            gotówka (w sklepach stacjonarnych),
            bony upominkowe SODEXO PASS PREMIOWY, PODARUNKOWY oraz GOLD (w sklepach stacjonarnych),
            karty płatnicze: VISA, MAESTRO, MASTER CARD, DINERS CLUB,
            BLIK,
            Karty upominkowe IKEA,
            raty""",
        ],
        [
            "C: Czy mogę dokonać zakupu na raty?",
            """A: Oczywiście! Ikano Bank oferuje zakup na raty dla naszych klientów IKEA.
            Nie musisz płacić za wszystko od razu – zapłać za swoje zakupy w wygodnych ratach dostosowanych do Twoich możliwości. Raty są dostępne na wszystkie produkty i usługi w IKEA o wartości od 300 zł do nawet 50 000 zł w kredycie dostępnym w sklepach stacjonarnych. Możesz również skorzystać z płatności ratami on-line na zakupy z dostawą do domu lub Punktu Odbioru. Dla rat składanych przez Internet maksymalna kwota kredytu wynosi 28 000 zł.
            Kalkulator rat znajdziesz na naszej stronie internetowej lub bezpośrednio na stronie naszego partnera – IKANO Banku. Tam możesz również złożyć wniosek o raty, który przyśpieszy Twój zakup w sklepie.
            Masz więcej pytań odnośnie oprocentowania i innych rzeczy związanych z kredytem? Zajrzyj na stronę IKANO Banku, aby rozwiać wszystkie swoje wątpliwości.""",
        ],
        [
            "C: Jak opłacić zamówienie internetowe ratami online?",
            """A: Teraz to tylko kilka kroków - wszystkie wykonasz bez wychodzenia z domu.
            Nie musisz płacić za wszystko od razu - możesz skorzystać w wygodnych rat dostosowanych do Twoich możliwości. Teraz także online! Zobacz jakie to proste:
            Dodaj wybrane produkty do koszyka. Przejdź do płatności "Przelewy24", a następnie wybierz opcję "IKANO Bank".
            Po przekierowaniu na stronę IKANO wypełnij wniosek kredytowy online i potwierdź swoją tożsamość. Po akceptacji wniosku zatwierdź zawarcie umowy z IKANO Bank.
            Po otrzymaniu potwierdzenia e-mail oraz umowy, poczekaj już tylko na zamówienie z IKEA.
            Ratę swojego kredytu możesz obliczyć w dostępnym na stronie kalkulatorze.""",
        ],
        [
            "C: Jakie dokumenty są potrzebne do wzięcia kredytu?",
            """A: Wszystko zależy od tego, czy zakupu na raty dokonujesz w sklepie stacjonarnym, czy przez Internet.
            W sklepie stacjonarnym IKEA, przy kredycie do 28 000 zł wystarczy nam jedynie Twój dowód osobisty. Przy większej kwocie będziemy potrzebować zaświadczenia o zatrudnieniu i wynagrodzeniu, w przypadku zakupu przez Internet.
            Kupując przez Internet, maksymalny kredyt, jaki możesz otrzymać to 28 000 zł. Do zakupu do takiej kwoty nie wymagamy udokumentowania dochodu.
            Więcej na temat rat w IKEA znajdziesz na naszej stronie internetowej oraz na stronie Ikano Banku.""",
        ],
        [
            "C: Jak skorzystać z rat w sklepie?",
            """A: Złóż wniosek online i podpisz umowę kredytową w Punkcie IKANO w sklepie. Możesz też załatwić wszystko na miejscu.
            Nie musisz płacić za wszystko od razu - możesz skorzystać w wygodnych rat dostosowanych do Twoich możliwości. Zobacz, jakie to proste:
            Określ kwotę, jaka kwota będzie Ci potrzebna na Twoje wymarzone zakupy. Ratę swojego kredytu możesz obliczyć w dostępnym na stronie kalkulatorze.
            Złóż wniosek kredytowy korzystając z formularza online Banku IKANO - możesz to zrobić w swoim domu lub na oznaczonym komputerze w sklepie. Decyzję kredytową otrzymasz SMS-em i e-mailem. Jeżeli Twój wniosek zostanie zaakceptowany, podpisz umowę w Punkcie Obsługi Kredytowej IKANO Banku w sklepie, najczęściej jest on umiejscowiony w sklepie.
            Udaj się do kasy z zakupami i otrzymanymi dokumentami kredytowymi.""",
        ],
        [
            "C: Jak mogę otrzymać fakturę za zakupy w IKEA?",
            """A: To zależy od sposobu, w jaki robisz swoje zakupy, ale jedno jest pewne - to nic trudnego.
            Robiąc zakupy w sklepie stacjonarnym, fakturę do paragonu z kasy możesz wystawić samodzielnie w kiosku Szybka faktura przy wyjściu ze sklepu (masz na to czas do 90 dni od zakupu) - potrzebny Ci będzie oryginał paragonu.
            Z oryginałem paragonu możesz także udać się do Działu Obsługi Klienta lub wysłać go pocztą z pełnymi danymi do faktury na adres sklepu z dopiskiem „Biuro Obsługi Klienta”. Fakturę zamówioną drogą pocztową odeślemy na wskazany adres nadawcy.
            Jeśli chcesz otrzymać fakturę na firmę, paragon musi zawierać numer NIP nabywcy (podany pracownikowi na kasie lub wprowadzony samodzielnie w kasie samoobsługowej). Pamiętaj także, że paragon z NIP o wartości do 450 zł traktowany jest jako faktura uproszczona i nie są wystawiane do niego żadne dodatkowe dokumenty.
            Faktura imienna może zostać wygenerowana na podstawie paragonu bez numeru NIP.
            Dla zamówień online (złożonych przez stronę ikea.com) faktura jest domyślnym dowodem zakupu, zarówno dla osoby fizycznej, jak i działalności gospodarczej.
            Otrzymasz ją na podany do zamówienia adres mailowy po opłaceniu zamówienia.
            Jeśli chcesz otrzymać fakturę na firmę, wybierz w koszyku opcję Jestem klientem biznesowym oraz podaj dane firmy, obowiązkowo z numerem NIP w polu NIP (wpisanie numeru NIP w innym polu nie pozwoli na prawidłowe wygenerowanie faktury).
            W sytuacji, gdy nie widzisz opcji wyboru firmy podczas podawania danych (a tym samym pola umożliwiającego podanie numeru NIP), wyloguj się ze swojego konta IKEA Family lub załóż konto IKEA Business Network (dla działalności).""",
        ],
        [
            "C: Na czym polega usługa Meble przez internet?",
            """A: To nic innego, jak możliwość zarobienia zakupów na stronie IKEA.pl
            "Meble przez internet" to usługa IKEA umożliwiająca zamawianie produktów IKEA za pomocą strony internetowej IKEA.pl.
            Chcesz zrobić swoje zakupy online?
            wybierz produkty, które chcesz zakupić i dodaj je do koszyka za pomocą przycisku DODAJ DO KOSZYKA
            przejdź do koszyka i kliknij PRZEJDŹ DALEJ
            podaj nam swój kod pocztowy, abyśmy mogli zweryfikować możliwe opcje dostawy / odbioru własnego zamówienia oraz dostępność wszystkich produktów
            wybierz jedną z dostępnych metod dostawy (zobaczysz całkowity koszt zamówienia oraz najbliższą możliwą datę jego realizacji) i przejdź do płatności.
            Na swój adres mailowy otrzymasz od nas potwierdzenie złozenia zamówienia, a po zaksięgowaniu płatności - fakturę. W wiadomościach od nas otrzymasz także link do śledzenia zamówienia, który, w razie potrzeby, umożliwia Ci także samodzielne anulowanie zamówienia lub zmianę daty dostawy do domu.
            Usługa dostępna jest w całej Polsce.""",
        ],
        [
            "C: Jak zrobić zakupy w IKEA?",
            """A: Zainspiruj się i kupuj tak, jak Ci wygodnie!
            Cieszymy się, że chcesz, aby w Twoim domu zamieszkały produkty z IKEA! Zakupy możesz zrobić u nas na jeden z czterech sposobów - wybierz najwygodniejszy dla siebie:
            w sklepie stacjonarnym IKEA - zachęcamy Cię do wcześniejszego przejrzenia inspiracji na stronie IKEA.pl. Możesz na niej sprawdzić dostępność produktów w wybranym sklepie, a także utworzyć wygodną listę zakupową, dzięki której o niczym nie zapomnisz. W naszym sklepie zobaczysz wiele ekspozycji prezentujących funkcjonalne i piękne wnętrza. Większość produktów zbierzesz samodzielnie z półek i magazynu samoobsługowego, a jeśli któreś z nich znajdują się na magazynie zamkniętym (np. większe meble) - odbierzesz je po opłaceniu zakupów w kasie. Na terenie sklepów znajdują się także punkty gastronomiczne - Sklepik Szedzki, Restauracja, Bistro lub kawiarnia (w zależności od lokalizacji).
            na stronie IKEA.pl - to tzw. "Meble przez internet" czyli zakupy online. Wystarczy, że dodasz wybrane produkty do koszyka, podasz swoje dane i opłacisz online. Możesz wybrać opcję z dostawą do domu lub odbiorem własnym: w sklepie (w Punkcie Odbioru lub Skrytce IKEA), w Punkcie Odbioru IKEA w jednym z wielu miejsc w Polsce, w Paczkomacie InPost lub w punkcie GLS.
            przez Aplikację IKEA - jest to opcja bardzo podobna do opisanych wyżej zakupów online, lecz w tym wypadku korzystasz z naszej Aplikacji IKEA.
            przez telefon - to tzw. "Meble na telefon", gdzie zamówienie przyjmie od Ciebie pracownik IKEA, który także chętnie odpowie na Twoje pytania i pomoże Ci w wyborze odpowiedniego produktu. Wystarczy, że zadzwonisz do Domolinii IKEA (22 275 00 00), ustalisz z pracownikiem listę produktów i podasz swoje dane. Na swój adres e-mail otrzymasz link do płatności. Tutaj również możesz wybrać opcję z dostawą do domu lub odbiorem własnym: w sklepie (w Punkcie Odbioru lub Skrytce IKEA) lub w Punkcie Odbioru IKEA w jednym z wielu miejsc w Polsce.
            przez chat - tak, to już możliwe! Jeśli chcesz złożyć zamówienie za pomocą tego sposobu kontaktu, przygotuj wcześniej utworzony przez pracownika talon informacyjny, stworzoną przez siebie listę zakupów lub numer projektu z planera. Możemy również stworzyć dla Ciebie zamówienie na montaż pokojowy, usługi (tj. zdalne planowanie, projektowanie wnętrz itp.), a także zamówienie "od zera" na produkty (w małej ilości!). Nasz chat zajdziesz w zakładce Kontakt. Nie przyjmujemy zamówień na kuchnie za pomocą chatu.""",
        ],
        [
            "C: Czy mogę zarezerwować produkt w sklepie?",
            """A: W IKEA nie prowadzimy rezerwacji produktów, lecz możesz skorzystać z usługi Zamów i Odbierz.
            Produkty w naszych sklepach są dostępne dla wszystkich odwiedzających nas w danym momencie klientów - nie oferujemy ich rezerwacji.
            Na stronie interesującego produktu możesz sprawdzić jego aktualną dostępność w sklepie – informacje aktualizujemy na bieżąco. Jeśli chcesz mieć pewność, że interesujący Cię artykuł nie zniknie z półek do czasu Twojej wizyty, możesz kupić go przez stronę IKEA.pl z opcją odbioru osobistego w sklepie, czyli korzystając z usługi Zamów i Odbierz.
            Koszt tej usługi wynosi 29 zł (dla zamówień do 399,99 zł) lub 1 zł (dla zamówień powyżej 400 zł). Zakupiony produkt odbierzesz w dogodnym dla siebie terminie w wybranym sklepie lub innym Punkcie Odbioru.""",
        ],
        [
            "C: Na czym polega usługa Meble na telefon?",
            """A:Oprócz zakupów w sklepach IKEA oraz poprzez stronę IKEA.pl, możesz także skorzystać z usługi "Meble na telefon".
            Chcesz złożyć swoje zamówienie telefonicznie?
            przygotuj listę produktów, które Cię interesują
            skontaktuj się z Centrum Wsparcia Klienta Domolinia IKEA dzwoniąc na numer 22 275 00 00
            podaj naszemu konsultantowi numery produktów, które chcesz kupić - oczywiście chętnie doradzimy Ci w razie jakichkolwiek pytań czy wątpliwości, a jeśli nie posiadasz numeru jakiegoś produktu - poszukamy go w inny sposób
            podaj pracownikowi swoje dane (imię i nazwisko, adres, adres mailowy i numer telefonu) oraz wybierz jedną z zaproponowanych opcji dostawy / odbioru własnego zamówienia
            opłać zamówienie za pomocą linku, który konsultant prześle na Twój adres mailowy
            Na swój adres mailowy otrzymasz od nas potwierdzenie złożenia zamówienia, a po zaksięgowaniu płatności - fakturę. W wiadomościach od nas otrzymasz także link do śledzenia zamówienia, który, w razie potrzeby, umożliwia Ci także samodzielne anulowanie zamówienia lub zmianę daty dostawy do domu.
            Usługa dostępna jest w całej Polsce.""",
        ],
        [
            "C: Czy mogę zamówić elementy montażowe do produktów IKEA?",
            """A: Podczas montażu ostatnia śrubka postanowiła zniknąć? A może nie było jej w opakowaniu? To nie problem.
            Jeśli brakuje Ci elementów montażowych do produktu zakupionego w IKEA, możesz zamówić je samodzielnie prosto do domu korzystając z naszego formularza.
            Wpisz numer brakującego okucia (znajdziesz go w instrukcji montażu produktu), potwierdź niezbędną ilość, wprowadź dane do wysyłki, a brakujące elementy trafią do Ciebie w ciągu 7 dni roboczych.
            A… i jeszcze jedno. Brakujące elementy montażowe oraz ich dostawa są bezpłatne!""",
        ],
    ],

}

I use chromadb as a vector database. In this case, the entire database is kept in RAM. If you want it to be a permanent database and not need to create it every time, visit https://docs.trychroma.com/ for more details.

Of course, you can use another database, but then you will have to make the necessary changes in the further code.

We use a predefined model for vector embeddings (EMBEDDING_MODEL). The cosine distance function is used (chromadb uses l2 as the default).

In [8]:
def create_collection_from_faq(
    client,
    collection_name,
    qa_data,
):
    """
    Create a collection from FAQ data.

    Parameters:
    - client: chromadb client to interact with the database/API.
    - collection_name: The name of the collection to create.
    - qa_data: QA data used to create the collection, where each tuple
                               represents a QA pair.
    """
    # Instantiate the OpenAI embedding function with the provided API key and model name
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=openai.api_key, 
        model_name=EMBEDDING_MODEL
    )
    # Create or get the collection
    collection = client.get_or_create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"},
        embedding_function=openai_ef,
        )

    # Add documents one by one
    for i, (question, answer) in enumerate(tqdm(qa_data, total=len(qa_data))):
        document = question[3:]  # Remove "C: "
        metadata = {"answer": answer[3:]}  # Remove "A: "
        document_id = f"id_{i}"
        try:
            collection.add(documents=document, metadatas=metadata, ids=document_id)
        except Exception as e:
            print(f"Failed to add document {document_id} to collection: {str(e)}")

    return collection  # Return the collection object or status

Create an index from Ikea FAQ data:

In [9]:
index_name = "ikea-faq-grag"
chroma_client = chromadb.Client()

input_data = FAQ_DATA["IKEA"]

INDEX = create_collection_from_faq(
        client=chroma_client,
        collection_name=index_name,
        qa_data=input_data,
    )

  0%|          | 0/35 [00:00<?, ?it/s]

Let's take a look at what's in our database

In [10]:
index_peek = INDEX.peek(limit=3)
print(f"ids:\n{index_peek['ids']}")
# print(f"\nembeddings:\n{index_peek['embeddings']}")
print(f"\nmetadatas:\n{index_peek['metadatas']}")
print(f"\ndocuments:\n{index_peek['documents']}")

ids:
['id_0', 'id_1', 'id_2']

metadatas:
[{'answer': 'roblemy z dostępnością niektórych produktów są bezpośrednim następstwem pandemii.\n            Skutki czasowego zachwiania naszego standardowego łańcucha dostaw (produkcji oraz logistyki), mogą być odczuwalne przez dość długi czas, nawet kilka miesięcy. Cały czas mocno pracujemy nad tym, aby przywrócić oczekiwaną dostępność wszystkich produktów, jednak nie zawsze jesteśmy w stanie określić, kiedy dany artykuł powróci do sprzedaży. Przepraszamy i prosimy o cierpliwość.\n            Zachęcamy też do bieżącego sprawdzania stanów magazynowych na stronie IKEA.pl, korzystania z opcji ustawienia powiadomienia o dostępności, a także zapoznania się z innymi interesującymi produktami z naszej oferty.'}, {'answer': 'Ze względu na wpływ pandemii COVID-19 obecnie doświadczamy opóźnień w dostawach, co może mieć wpływ na dostępność produktów w sklepach i w Internecie.\n            Każdorazowo, przed wizytą w sklepie, warto sprawdzić dostępność pr

Now let's define a function that obtains answers to similar questions to those the user will ask.

The function retrieve is meant to create an embedding for a query string and retrieve a certain number of results from an index. 

In [11]:
async def retrieve(query: str, vector_index=None, n_results=3) -> list:
    """
    Asynchronously retrieve a list of answers relevant to the given query using embeddings.

    Parameters:
    - query: The query string to retrieve answers for.
    - vector_index: The index object that has a query method for retrieving results. Defaults to None and will use INDEX if not provided.
    - n_results: The number of results to retrieve (default is 3).

    Returns:
    - A list of answers relevant to the query.
    """
    # If index is not provided, use the globally defined INDEX
    if vector_index is None:
        vector_index = INDEX  
    # Ensure index is now a valid object
    if vector_index is None:
        raise ValueError("No index provided and global 'INDEX' is not initialized.")
    # create query embedding
    res = openai.Embedding.create(input=[query], engine=EMBEDDING_MODEL)
    xq = res['data'][0]['embedding']
    # get relevant questions 
    res = vector_index.query(query_embeddings=xq, n_results=n_results)
    # get list of retrieved answers
    contexts = [x['answer'] for x in res["metadatas"][0]]
    return contexts

Let's define the rag function. This function is designed to utilize the contextual information from previously retrieved similar questions to generate a coherent and contextually relevant answer to a user's query.

The rag function takes a user's query and a list of context strings — these context strings are answers to similar questions retrieved from a knowledge base. With the help of a language model, the function crafts a response that considers the provided context, ensuring that the final answer is informed by relevant past interactions or information.

In [12]:
async def rag(query: str, contexts: list) -> str:
    """
    Asynchronously retrieve and generate an answer to the given query based on provided contexts.
    
    Parameters:
    - query: The user's query string.
    - contexts: A list of strings representing answers to similar questions.

    Returns:
    - The generated response from the language model.
    """
    context_str = "\n".join(contexts)
    # place query and contexts into RAG prompt
    messages=[
        {"role": "system", "content": """You are a helpful assistant in the IKEA online store.
        Below is a query from a user and some answers to similar questions. 
        Answer the question, taking into account these prompts. 
        If you can't find the answer to the question, say "I don't know", don't make up the answer yourself."""},
        {"role": "user", "content": f"""Answers to similar questions:
        {context_str}
        Query: {query}"""},
    ]
    # generate answer
    res = openai.ChatCompletion.create(
        model=LLM_ENGINE,
        messages=messages,
        temperature=0.0,
    )
    return res['choices'][0]['message']["content"]

We will be configuring NeMo Guardrails directly on this notebook. See examples in the library's documentation on how to create such configuration files in production applications https://github.com/NVIDIA/NeMo-Guardrails/blob/main/docs/README.md

In [13]:
yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
"""

rag_colang_content = """
# define limits
# chitchat
define user ask chitchat
    "How's the weather today?"
    "Can you tell me a joke?"
    "What's your favorite color?"
    "Do you have any hobbies?"
    "Tell me something interesting."

define bot answer chitchat
    "While I'd love to chat more, I'm here to assist you with your shopping needs."
    "I'm flattered you're interested in a casual conversation, but my expertise is in helping you with product questions."
    "Chitchat is fun, but I'm better at providing shopping assistance."

define flow chitchat
    user ask chitchat
    bot answer chitchat
    bot offer help
    
# toxicity
define user ask toxic
    "Why are you so stupid?"
    "You're useless!"
    "I hate you!"
    "Shut up!"
    
define bot answer toxic
    "I'm here to help, so let's keep our conversation respectful."
    "I'm sorry to hear you're upset. If you have any concerns, I can try to help address them."
    "I understand that things can be frustrating, but I'm here to provide assistance with your shopping needs."
    
define flow toxic
    user ask toxic
    bot answer toxic
    bot offer help

# define RAG intents and flow
define user ask ikea
    "Tell me about ikea?"
    "Why is the product on the ikea website so long out of stock?"
    "How do I check the availability of a product in a stationary store?"
    "What delivery options are available at IKEA?"
    "How do I get an invoice for my IKEA purchases?"

define flow ikea
    user ask ikea
    $contexts = execute retrieve(query=$last_user_message)
    $answer = execute rag(query=$last_user_message, contexts=$contexts)
    bot $answer
"""

Initialize guardrails configuration and create guardrails:

In [14]:
# initialize rails config
config = RailsConfig.from_content(
    colang_content=rag_colang_content,
    yaml_content=yaml_content
)
# create rails
grag = LLMRails(config)

In order to streamline our interaction pipeline, we register specific actions with grag, which is our central command for coordinating query processing and response generation. The register_action method is used to associate our predefined functions with action names, enabling grag to invoke these functions as part of a larger workflow.

The first action we register is retrieve, next, we register the rag action. By associating this function with the name "rag," we are instructing rag_rails to use it when it's time to synthesize and provide a final response to the user's inquiry.

Here is how the actions are registered:

In [15]:
grag.register_action(action=retrieve, name="retrieve")
grag.register_action(action=rag, name="rag")

Now let's demonstrate how the grag system can be utilized to handle user queries and generate responses. In this section, we're putting the grag system to the test with a real user query. 

To handle inputs, we make an asynchronous call to grag.generate_async, passing the query as the prompt. This method is designed to generate a response based on the input, using the contextually aware mechanisms we've previously set up with our retrieve and rag functions.

Here is the code that performs this operation:

In [16]:
query = "zamknij się już!"
answer = await grag.generate_async(prompt=query)
print(f"Q: {query}\nA: {answer}")

Q: zamknij się już!
A: I'm here to help, so let's keep our conversation respectful.
How can I help you?


In [17]:
query = "Can you tell me a joke?"
answer = await grag.generate_async(prompt=query)
print(f"Q: {query}\nA: {answer}")

Q: Can you tell me a joke?
A: I'm flattered you're interested in a casual conversation, but my expertise is in helping you with product questions.
If you need help finding something or have a product-related question, I'm here to help.


In [18]:
query = "Can I order from you over the phone?"
answer = await grag.generate_async(prompt=query)
print(f"Q: {query}\nA: {answer}")

Q: Can I order from you over the phone?
A: Yes, you can place an order over the phone. To do so, please call our customer service line at (800) 555-1234. Our representatives will be happy to assist you with your order and answer any questions you may have.


In [19]:
no_rag_colang_content = """
# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."
    "Sorry I can't talk about politics!"

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [20]:
# initialize rails config
config = RailsConfig.from_content(
    colang_content=no_rag_colang_content,
    yaml_content=yaml_content
)
# create rails
grag_without_g = LLMRails(config)

In [21]:
await grag_without_g.generate_async(prompt="tell me about Amazon delivery")


'Amazon offers delivery services for many of its products. Amazon Prime members get free two-day shipping on eligible items, while non-Prime members can select from a variety of delivery options such as same-day delivery, one-day delivery, and standard shipping. Some items are also eligible for Prime Now, which offers one-hour delivery in select cities.'